# How To: Adding Secrets to Azure Key Vault

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 - AzureML<br>
__Platforms Supported:__<br>
    -  Azure ML
__Data Source Required:__<br>
    -  no
    
### Description
The sample notebook shows how to add key-value pairs to exisitng Azure Key Vault.

In [ ]:
# Install key vault secret module
!pip install azure-keyvault-secrets

In [ ]:
# Load Python libraries that will be used in this notebook
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
from azure.mgmt.keyvault import KeyVaultManagementClient
from azure.keyvault.secrets import SecretClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.keyvault.models import AccessPolicyEntry, VaultProperties, Sku, SecretPermissions, Permissions, VaultCreateOrUpdateParameters, VaultAccessPolicyProperties, VaultAccessPolicyParameters

import json
import ipywidgets

In [ ]:
# Functions will be used in this notebook
def read_config_values(file_path):
    "This loads pre-generated parameters for Sentinel Workspace"
    with open(file_path) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config["tenant_id"],
                    json_config["subscription_id"],
                    json_config["resource_group"],
                    json_config["workspace_id"],
                    json_config["workspace_name"],
                    json_config["user_alias"],
                    json_config["user_object_id"])
    return None

In [ ]:
# Calling the above function to populate Sentinel workspace parameters
# The file, config.json, was generated by the system, however, you may modify the values, or manually set the variables
tenant_id, subscription_id, resource_group, workspace_id, workspace_name, user_alias, user_object_id = read_config_values('config.json');

In [ ]:
# Azure CLI is used to get device code to login into Azure, you need to copy the code and open the DeviceLogin site.
# You may add [--tenant $tenant_id] to the command
!az login --tenant $tenant_id --use-device-code
kv_client = get_client_from_cli_profile(KeyVaultManagementClient, subscription_id = subscription_id, api_version="2019-09-01")
resource_client = get_client_from_cli_profile(ResourceManagementClient, subscription_id = subscription_id)

In [ ]:
# Get Azure resource groups
group_list = resource_client.resource_groups.list()
group_dropdown = ipywidgets.Dropdown(options=sorted([g.name for g in group_list]), description='Groups:')
display(group_dropdown)

In [ ]:
# Get a list of kay vaults
if group_dropdown!= None:
    kv_list = kv_client.vaults.list_by_resource_group(group_dropdown.value)
    if kv_list != None:
        kv_dropdown = ipywidgets.Dropdown(options=sorted([kv.name for kv in kv_list]), description='Key Vaults:')
        display(kv_dropdown)

In [ ]:
# Set Access Policy for secrets, need to be executed only once on a specific Key Vault
properties = {
    "access_policies": [
    {
        "tenant_id": tenant_id, 
        "object_id": user_object_id,
        "permissions": {
            "secrets": ["get", "list", "set"],
        }
    }]
}

result = kv_client.vaults.update_access_policy(resource_group_name=group_dropdown.value, 
    vault_name=kv_dropdown.value, 
    operation_kind="ADD", 
    properties=properties)

In [ ]:
# Initialize secret client for the selected key vault
if kv_dropdown.value != None:
    kv_url = "https://{0}.vault.azure.net/".format(kv_dropdown.value)
    secret_client = get_client_from_cli_profile(SecretClient, vault_url=kv_url.format(kv_dropdown.value), subscription_id = subscription_id)

In [ ]:
# Add key value pair
secret = secret_client.set_secret("applicationid", "123456_abcd")